### 1. Подготовка

Начинаем с импорта необходимых библиотек и определения ключевых параметров для сбора данных о товарах с [онлайн-магазина «Пятёрочки»](https://5ka.ru/). Сюда входят базовые URL-адреса для отправки запросов к API магазина, заголовки, имитирующие запрос из браузера, а также текущая дата, чтобы маркировать файлы.

In [ ]:
import requests
import json
import time
import random
import pandas as pd
from datetime import date

# базовые URL-адреса и заголовки для запросов к сайту
BASE_URL = "https://5d.5ka.ru/api/catalog/v2/stores/Y233/categories/"
CATEGORIES_URL = "https://5d.5ka.ru/api/catalog/v2/stores/Y233/categories?mode=delivery&include_subcategories=1"
USER_AGENT = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/132.0.0.0 Safari/537.36"
CHECK_DATE = date.today() # текущая дата для маркировки файлов

HEADERS = {
    "user-agent": USER_AGENT,
    "origin": "https://5ka.ru"
}

### 2. Веб-скрейпинг

Чтобы получить данные о товарах с сайта «Пятёрочка», мы выполняем следующие шаги: сначала получаем список категорий, затем собираем данные о товарах по каждой категории, очищаем сырые данные и сохраняем результат в структурированном виде.

#### 2.1. Получение категорий

Поскольку товары сгруппированы по категориям, сначала необходимо получить список всех доступных категорий и подкатегорий. Этот список запрашивается со специального адреса, используемого для отображения каталога на сайте. Каждая подкатегория содержит уникальный буквенно-цифровой идентификатор, который позже используется для получения списка соответствующих товаров.

Мы определяем функцию, которая обращается к этому адресу, извлекает только нужные поля (*id*, *name* и *parent_id*) и возвращает очищенный список подкатегорий.

In [ ]:
def fetch_categories():
    
    response = requests.get(CATEGORIES_URL, headers=HEADERS) # получаем данные о категориях от API
    raw_categories = json.loads(response.text) # преобразуем JSON-ответ в список словарей с категориями
    
    cleaned_categories = []
    
    # проходим по исходным данным и выбираем только нужные поля
    for category in raw_categories:    # проходим по каждой категории верхнего уровня
        for subcategory in category["categories"]:    # перебираем все подкатегории и сохраняем их, указывая родительскую категорию
            cleaned_categories.append({
                "id": subcategory["id"], # идентификатор, необходимый для последующей загрузки товаров
                "name": subcategory["name"], # название подкатегории (для справки)
                "parent_id": category["id"] # идентификатор родительской категории (для справки)
            })

    return cleaned_categories

Мы получаем все доступные категории и сохраняем их в CSV-файл — чтобы данные было удобно просматривать и при необходимости повторно использовать.

In [ ]:
categories = fetch_categories()
# categories = categories[83:] # эту строку можно использовать, чтобы продолжить сбор данных с определённого места в случае ошибки
total_categories_count = len(categories) # для отображения прогресса при сборе данных

categories_df = pd.DataFrame(categories)
categories_df.to_csv(f'categories-{CHECK_DATE}-pyaterochka.csv', index=False)

In [ ]:
# необязательный промежуточный вывод списка категорий
# with pd.option_context('display.max_rows', None, 'display.max_colwidth', None):
#     display(categories_df)

#### 2.2. Получение исходных данных о товарах

Сайт «Пятёрочки» динамически подгружает данные о товарах с помощью GET-запросов, привязанных к конкретным кодам категорий. У каждой категории есть уникальный буквенно-цифровой идентификатор, который передаётся в URL при запросе товаров. Сначала страница загружает 12 товаров, а затем постепенно увеличивает их количество (например, 24, 36, 48...) по мере прокрутки.   
Чтобы воспроизвести это поведение в нашем скрипте, мы увеличиваем параметр `limit` в каждом запросе и каждый раз пересохраняем пополняющийся список товаров, пока не перестанут появляться новые позиции. Между запросами добавляется случайная задержка, чтобы избежать ограничения по частоте запросов или блокировки.

На этом этапе мы получаем сырые данные о товарах в формате JSON, сохраняя их в том виде, в котором они представлены на сайте.

In [ ]:
def fetch_products(category_id):
    
    current_limit = 0
    total_products = 0
    products = []
    
    while total_products >= current_limit:
        
        time.sleep(random.uniform(1, 5)) # случайная задержка, чтобы избежать блокировки со стороны сервера
        
        current_limit += 12 # увеличиваем лимит, чтобы загрузить больше товаров
        url = f'{BASE_URL}{category_id}/products?mode=delivery&include_restrict=false&limit={current_limit}'
        
        response = requests.get(url, headers=HEADERS) # получаем данные о товарах для текущей категории и лимита
        response_data = json.loads(response.text) # преобразуем ответ в словарь
        
        products = response_data["products"] # извлекаем только список товаров
        total_products = len(products) # сохраняем общее количество товаров на текущий момент
        
        print(f'{len(products)}..', end='') # индикатор прогресса — выводим, сколько товаров получено на текущем шаге

    return products    

#### 2.3. Очистка данных о товарах

Далее мы определяем функцию, которая извлекает из сырых данных только необходимые поля:

- Идентификатор категории (для справки)
- Название товара
- Единица измерения
- Обычная и скидочная цена
- Уточнение по цене (например, вес нетто или единица цены)

В итоге получается очищенный список товаров, готовый к сохранению или дальнейшему анализу.

In [ ]:
def clean_product_data(category, raw_products):

    cleaned_products = []

    # проходим по сырым данным и выбираем только нужные поля
    for product in raw_products:
        cleaned_products.append({
            "category_id": category, # идентификатор категории
            "name": product["name"], # название товара
            "unit_of_measurement": product["uom"], # единица измерения
            "price_reg": product["prices"]["regular"], # обычная цена
            "price_disc": product["prices"]["discount"], # скидочная цена
            "pricing_clarification": product["property_clarification"] # уточнение по цене
        })

    return cleaned_products

#### 2.4. Сводим все вместе

В основном цикле мы:

1. Проходим по выбранным категориям,
2. Получаем и очищаем данные о товарах для каждой из них,
3. Добавляем очищенные данные из каждой категории в общий CSV-файл, собирая полный набор данных.

Для удобства отслеживания в процессе выводится количество загруженных товаров по каждой категории. Чтобы избежать блокировок и ограничений по частоте запросов, между запросами добавляется небольшая случайная задержка. В имя файла включается текущая дата (`CHECK_DATE`), чтобы зафиксировать, когда данные были выгружены.

In [ ]:
fetched_categories_count = 0    # счётчик категорий для отслеживания прогресса загрузки

# создаём файл с заголовком таблицы для записи данных
products_df = pd.DataFrame(columns=["category_id", "name", "unit_of_measurement", "price_reg", "price_disc", "pricing_clarification"])
products_df.to_csv(f'scraped_products-{CHECK_DATE}-pyaterochka.csv', index=False, mode='w')

for category in categories:
    
    raw_products = fetch_products(category["id"]) # загружаем товары
    new_products = clean_product_data(category["id"], raw_products) # очищаем данные, оставляя только нужные поля

    products_df = pd.DataFrame(new_products)
    products_df.to_csv(f'scraped_products-{CHECK_DATE}-pyaterochka.csv', index=False, mode='a', header=False) # добавляем данные в CSV

    fetched_categories_count += 1
    print(f'Категория {category["id"]} обработана, загружено {fetched_categories_count} из {total_categories_count} категорий')
    
    time.sleep(random.uniform(1, 5))

print(f'Загрузка завершена. Результаты сохранены в файле scraped_products-{CHECK_DATE}-pyaterochka.csv')

### 3. Фильтрация и нормализация данных о товарах

После сбора и очистки исходных данных о товарах переходим к фильтрации — оставляем только те товары, которые подходят для сравнения. Этот этап состоит из нескольких шагов:

#### 3.1. Первичная обработка

Начинаем с загрузки ранее сохранённых данных о товарах и категориях, удаляем ненужные поля. Для удобства дальнейшей работы упрощаем названия полей. Также убираем точные дубли — они могут появиться, если один и тот же товар был в нескольких категориях.

In [ ]:
import pandas as pd
import re

categories = pd.read_csv('categories-2025-03-03-pyaterochka.csv')
products_original = pd.read_csv('scraped_products-2025-03-03-complete.csv')

# удаляем ненужные поля и упрощаем названия
products = products_original.drop(['category_id', 'price_disc', 'unit_of_measurement'], axis=1) # удаляем поля category_id, price_disc
                                                                                                # и unit_of_measurement
products = products.rename(columns={'price_reg': 'price', 'pricing_clarification': 'pricing_unit'}) # переименовываем ценовые поля для удобства
products = products.drop_duplicates() # удаляем дубли

#### 3.2. Фильтрация товаров по типу

Чтобы выбрать из набора только релевантные для сравнения товары, мы создаём словарь с типами продуктов и соответствующими регулярными выражениями. Каждое выражение выбирает базовую форму продукта, исключая вариации, которые не входят в сферу нашего анализа.

> **Примечание:** приведенные здесь регулярные выражения не универсальны и подобраны под конкретный набор данных.

In [ ]:
product_regex_map = {
    'rice': r'(^|^")рис\b',
    'bread': r'(^хлеб\b|^багет\b|^батон\b)(?!.*(чесн|заморож))', # выбирает "хлеб", "багет", или "батон"
                                                                 # но исключает "багет с чесноком" или "багет замороженный"
    'chicken_fillet': r'^филе\b.*(кур|цыпл)(?!.*запеч)',
    'pork_leg': r'^окорок\b.*свин',
    'egg': r'^яйцо.*курин',
    'cucumber': r'^огур(цы|ец)(?!.*(солен|маринован))', # выбирает "огурец" или "огурцы", но исключает "огурцы соленые" или "огурцы маринованные"
    'carrot': r'^морковь(?!.*корей)',
    'onion': r'^лук.*реп(?!.*зелен)',
    'tomato': r'^томаты(?!.*(сок|очищ|маринован|вялен|солен))',
    'cabbage': r'^капуста\b.*белокоч',
    'eggplant': r'^баклажаны?($|.*теплич)',
    'banana': r'^банан',
    'orange': r'^апельсин',
    'milk': r'^молоко(?!.*(сгущ|сух))',
    'yogurt': r'^йогурт\b(?!.*питье)', # выбирает "йогурт", но исключает "йогурт питьевой"
    'condensed_milk': r'(^молоко.*сгущ|^сгущ)(?!.*(варен|какао|шокол))',
    'green_tea': r'^чай.* зел(?!.*(порош|л$))',
    'black_tea': r'^чай.* черн(?!.*л$)',
    'ground_coffee': r'^кофе(?!.*(капсул|раствор)).*молот',
    'sugar': r'^сахар\b(?!.*ванил)',
    'salt': r'^соль(?!.*(розов|посуд|чесн|ванн|спец))',
    'sunflower_oil': r'^масло\b.*подсолн(?!.*добавл)',
    'water': r'^вода(?!.*(малин|лимон)).*негаз',
    'buckwheat': r'(^крупа\b.*гречн|^гречка\b)(?!.*(\bпшен|\bкиноа))',
    'spaghetti': r'(^макароны.*спагетти|^спагетти\b)(?!.*(заморож|кукуруз))',
    'rice_noodles': r'(^лапша|^вермишель).*(фунчоз)',
    'tofu': r'^тофу\b',
    'mango': r'^манго\b(?!.*(суш|заморож))'
}
product_regex_list = '|'.join(product_regex_map.values()) # объединяем все регулярные выражения в одно, используя оператор OR (|)

# фильтруем товары, названия которых соответствуют одному из выбранных типов продуктов
filtered_products = products.loc[products.name.str.contains(product_regex_list, case=False, regex=True)]

#### 3.3. Присвоение меток

Каждому отфильтрованному товару присваивается метка с типом продукта на основе совпадения по регулярному выражению. Также для удобства отслеживания указывается название супермаркета («Pyaterochka»).

In [ ]:
def assign_product_type(row):
    name = row['name']
    for product_type, regex in product_regex_map.items():
        match = re.search(regex, name, flags=re.IGNORECASE)
        if match:
            return product_type
    return None

filtered_products = filtered_products.copy()  # создаём копию, чтобы избежать предупреждений при добавлении новых полей через .loc
filtered_products.loc[:,'product_type'] = filtered_products.apply(assign_product_type, axis=1)
filtered_products.loc[:,'supermarket'] = 'Pyaterochka'

# необязательный промежуточный вывод списка отфильтрованных товаров
# with pd.option_context('display.max_rows', None, 'display.max_colwidth', None):
#     display(filtered_products)

#### 3.4. Извлечение и нормализация единиц измерения

Разные единицы товаров отличаются по количеству, весу или объёмы. Чтобы сравнение было корректным, мы извлекаем нужную информацию из названия товара или из поля с уточнением цены и рассчитываем нормализованные показатели — цену за килограмм, литр или за штуку.

- Вес в граммах
- Количество штук (в частности, яиц)
- Объём в миллилитрах

Каждое из значений извлекается по шаблону (через регулярное выражение). Не все товары содержат все эти значения, поэтому соответствующие колонки с нормализованными ценами (*price_kg*, *price_lit*, *price_unit*) могут остаться пустыми для некоторых товаров.

In [ ]:
def extract_weight(row):
    """Извлекает общий вес в граммах из названия товара или уточнения цены.
    Поддерживает как одиночный вес, так и формат с несколькими порциями (например, '5x100г').
    """
    
    name, pricing_unit = row['name'], row['pricing_unit']

    # формат с несколькими порциями (например, 5x100г)
    match = re.search(r'(\d+)(x|х)(\d+|\d+[.]\d+)\s?г', name) # выбирает (число) х (число) г
    if match:
        portion, per_portion = map(float, match.group(1,3)) # извлекаем количество порций и вес каждой
        return portion * per_portion # возвращаем общий вес
    # одиночный вес (в граммах или килограммах)
    match = re.search(r'(\d+|\d+[.]\d+)\s?(г|кг)', name)
    if match:
        weight = float(match.group(1))
        unit = match.group(2)
        return weight * 1000 if unit == 'кг' else weight # преобразуем килограммы в граммы
    # если в названии ничего нет, проверяем pricing_unit
    match = re.search(r'(\d+|\d+[.]\d+)\s?(г|кг)', pricing_unit)
    if match:
        weight = float(match.group(1))
        unit = match.group(2)
        return weight * 1000 if unit == 'кг' else weight

    return None  # если ничего не найдено

# следующие две функции работают по той же логике, что и extract_weight, но для штук и миллилитров
def extract_number_of_units(row):
    """Извлекает количество единиц ('шт') из названия товара или уточнения цены."""
    
    name, pricing_unit = row['name'], row['pricing_unit']

    # проверяем название
    match = re.search(r'(\d+)\s?шт', name)
    if match:
        number_of_units = int(match.group(1))
        return number_of_units
    # проверяем pricing_unit
    match = re.search(r'(\d+)\s?шт', pricing_unit)
    if match:
        number_of_units = int(match.group(1))
        return number_of_units

    return None  # если ничего не найдено

def extract_volume(row):
    """Извлекает общий объем в миллилитрах из названия товара или уточнения цены.
    Поддерживает как одиночный объем, так и групповую упаковку (например, '5x100мл').
    """
    
    name, pricing_unit = row['name'], row['pricing_unit']

    # формат с несколькими порциями (например, 5x100мл)
    match = re.search(r'(\d+)(x|х)(\d+|\d+[.]\d+)\s?мл', name)
    if match:
        portion, per_portion = map(float, match.group(1,3))
        return portion * per_portion
    # одиночный объем (в литрах или миллилитрах)
    match = re.search(r'(\d+|\d+[.]\d+)\s?(мл|л\b)', name)
    if match:
        volume = float(match.group(1))
        unit = match.group(2)
        return volume * 1000 if unit == 'л' else volume
    # проверяем pricing_unit
    match = re.search(r'(\d+|\d+[.]\d+)\s?(мл|л\b)', pricing_unit)
    if match:
        volume = float(match.group(1))
        unit = match.group(2)
        return volume * 1000 if unit == 'л' else volume

    return None  # если ничего не найдено

filtered_products = filtered_products.copy()  # пересоздаем датафрейм

# вычисляем нормализованные цены
filtered_products.loc[:,'weight'] = filtered_products.apply(extract_weight, axis=1)  # столбец с весом в граммах
filtered_products.loc[:,'price_kg'] = filtered_products.price / filtered_products.weight * 1000   # столбец с ценой за кг

filtered_products.loc[:,'number_of_units'] = filtered_products.apply(extract_number_of_units, axis=1)  # столбец с количеством штук
filtered_products.loc[:,'price_unit'] = filtered_products.price / filtered_products.number_of_units   # столбец с ценой за штуку

filtered_products.loc[:,'volume'] = filtered_products.apply(extract_volume, axis=1)  # столбец с объемом в мл
filtered_products.loc[:,'price_lit'] = filtered_products.price / filtered_products.volume * 1000   # столбец с ценой за литр

# необязательный промежуточный вывод
# with pd.option_context('display.max_rows', None, 'display.max_colwidth', None):
#     display(filtered_products)

#### 3.5. Сохранение финального отфильтрованного набора данных

Наконец, отфильтрованный и дополненный набор данных сохраняется в новый CSV-файл для дальнейшего анализа.

In [ ]:
filtered_products.to_csv(f'filtered_products-2025-03-03-pyaterochka.csv')